# Classical Statistical Inference: Bootstrap and Jacknife

## Table of Content

- II. What is statistical inference ? See [Frequentist_inference_01.ipynb](Frequentist_inference_01.ipynb)
    * II.3 Hypothesis testing See [Frequentist_inference_H_test.ipynb](Frequentist_inference_H_test.ipynb)
    * II.4 [Bootstrap](#II.4-Bootstrap)
       - [II.4.1 Bootstrap confidence interval](#Bootstrap-Confidence-Interval) 
       - [II.4.2 Jacknife](#II.4.2-Jacknife)     
    
- X. [References and supplementary material](#X)

## II.4 Bootstrap

The bootstrap is a method for estimating standard error (and confidence intervals on some statistical quantities / aka statistics) using your observed sample of random variable to estimate the population distribution of your statistics. 

Imagine that you have a random variable $x$, and have n measurements $\{x_i\}$ of it. Then you may want to measure some statistics (let's say $q = g(x_1, ..., x_n)$) of the pdf of $x$ (e.g. its mean, its median, its variance, ...) and find out how precise is that statistics (i.e. measure the variance or confidence interval associated to that stat.). In general, you do not know the distribution $h(x)$ from which are drawn your data, and therefore you do not know the one of $q$. You only have an estimate $\hat{h}(x) \, \equiv\, f(x)$:

$$
f(x) \, = \, \frac{1}{n} \, \sum_{i=1}^n \, \delta(x - x_i),  
$$

where $\delta(x)$ is the dirac function. This choice of $f(x)$ is also the one that maximizes the likelihood, i.e. $p(x_i  \, | \, f)$. Now that we have an estimator of $h(x)$ we can draw samples from $f(x)$ (as we did when we used Monte-Carlo to derive pdf for virtually infinite samples of random variables), and study the distribution of our statistical quantity (i.e. estimate its variance, confidence interval, ...). This is exactly as we did when we carried out Monte-Carlo simulations *except* that instead of drawing samples from a known pdf $h(x)$ we do it from our estimator of the pdf $f(x)$.    

If our sample contains $n$ elements, then, there are $n!$ possible distinct samples of size n we can draw , and a probability $n!/n^n$ that we draw a sample identical to the original data (i.e. for 10 elements, $n!/n^n \sim 3.6\,10^{-4}$.).  

In practice:
- You have: $x=\{x_1, x_2, ..., x_n\}$
- You then draw $B$ samples $\{x^*_1, x^*_2, ..., x^*_n\}$, where $x^*_i$ is randomly picked among the $x$ (with replacements, i.e. it could be that one of your $B$ samples will be $\{x_1, x_1, ...,x_1\}$ !). In other words, for each bootstrap sample, you generate random n indices in the range $[1, ..., n]$  (in `python`, you can use `np.random.randint(low, high, size)` or `scipy.random.randint()` to generate n random integers) to build your bootstrap sample from the original sample. 
- Then, as we did with Monte-Carlo simulations, you can estimate from each of these $B$ samples your statistics $\hat{q}$ and study its distribution, or more specifically derive its variance, confidence interval, ... 

**Note:** Sometimes, instead of using $f(x)$ as derived directly from the sample, one uses its "best fit model" $f(x \, | \, \hat{\theta})$, this is then called *parametric bootstrap*.  
There is complete books dedicated to bootstraps and its subtleties (especially if the errors on your random variable are not identical, effects related to the size of your sample, ... ). 

**Exercise:**

- Create a random sample $x$ of $N=100$ measurements drawn from a normal gaussian distribution: `sample = np.random.normal( 0, 1, 100)`. This is assumed to be your data set. Now, calculate a bootstrap estimate of the mean and the associated standard error (on the mean), assuming that *you do not know* that the population is normally distributed. 
- Calculate an estimator of the variance $\hat{\sigma}^2$ of your distribution and its associated (standard) error. 
- How do those values compare to theoretical expectation ? 

### II.4.1 Bootsrap confidence interval 

If we calculate a statistics $q$
There are many ways to derive confidence intervals using bootstrap: 

* -1.- Normal Interval: 
If you assume that your statistics follows a normal distribution (the interval is only accurate if this is true), 
$$
[\hat{q} - z_{\alpha/2} \, stde^B(q), \hat{q} + z_{\alpha2}\, stde^B (q)],
$$ 

where $\hat{\theta}$ is the estimator based on the original distribution, and $se^B$ is the bootstrap standard error. 

Similarly, one can also use the student's distribution to estimate the confidence interval, hence replacing $z_{\alpha2}$ by $t_{\alpha/2}$ 


For a 95% confidence interval, you have:
$$
[\hat{q} - 1.96 \, stde^B(q), \hat{q} + 1.96 \, stde^B (q)],
$$

* -2.- Percentile: You simply take the $\alpha/2$, $1-\alpha/2$ percentiles of the distribution of your $q^*_i$:    

$$
[q^*_{\alpha/2}, q^*_{1-\alpha/2}]
$$

If the bootstrap replicates of $\hat{q}^{*}$ follow a normal distribution $N(\hat{q}, stde^B (q))$, this percentile interval works reasonably well in many situations. It has also the property to be invariant to monotone transformations of $q$ (i.e. $m(q)$). 

* -3.- A third approach is the so-called "pivotal confidence intervals". Those are defined as:

$$
[2 \hat{q} - q^*_{1-\alpha/2}, 2 \hat{q} - q^*_{\alpha/2})],
$$

* Other approaches exist in the litterature. Only a "preliminary" form of bootstrap confidence interval estimate exists in python within the package `scikits.bootstrap`. Hence, you may need to develop your own functions for estimating bootstrap confidence interval. 

**Exercise**:

Calculate 95% CI for the sample used in the previous exercise and compare your results to theory (as your original sample is drawn from a normal distribution).

### II.4.2 Jacknife

The jacknife is a sort of bootstrap but in principle more robust (that claimed robustness gave it its name). The idea of the jacknife is to leave one observation aside for computing the statistic of interest. 

If $q_n$ is a statistical quantity estimated based on a sample of $n$ points. If we remove one data point, we can calculate $n$ times the statistics $q^*_i$ based on $n-1$ points (each time removing a *different* point).  
$q_{\infty}$ is the statistics we would have for an infinite sample. It can be shown (expanding $q_n$ as a series in $n^{-1}$, i.e. $q_n = q_{\infty} + n^{-1}\, q'_n + O(n^{-2})$ ; see [Lupton 1993](#LUP93)) that a bias corrected estimate $q^J$ can be computed as:

$$
q^J = q_n \, + \, \Delta\,q
$$

where $q_n$ is the estimator derived from the full sample and the jacknife correction $\Delta\,q$ is:

$$
\Delta \, q = (n-1)\, \left( q_n \, - \, \frac{1}{n}\, \sum_{i=1}^n \, q^*_i \right)
$$

Therefore, the bias-corrected estimate of q is simply:

$$
q^J = n \, q_n \, - \, (n-1) \, \bar{q}_n 
$$


In addition, if the estimator $q_n$ is asymptotically normal, then the variance for the jacknife estimate is:

$$
\sigma^2_q \, = \, \frac{n-1}{n} \sum^n_{i=1} (q^*_i \, - \, \bar{q}_n )^2
$$

with $\bar{q}_n = n^{-1} \, \sum^n_{i=1} \, q^*_i$  ($q^*_i$ is the stat where we have removed the $i$-th sample). The standard error $\hat{se}^J$ on the jacknife statisitcs $q^J$ is simply the sqrt of the variance $\sigma^2_q$.  

**Exercise**

For the same distribution as the one used in the bootstrap case, calculate the mean and standard error on the mean and compare to the bootstrap result.    
TIP: You can use `sample = np.delete(sample, index)` to remove `sample[index]` from sample.  

## X. References and supplementary material: <a class="anchor" id="X"></a>

**Chapter 4** (4.1 4.2, 4.5, 4.7) of the book <a class="anchor" id="book"></a> *Statistics, data mining and Machine learning in astronomy* by Z. Ivezic et al. in Princeton Series in Modern Astronomy. 

*All of statistics: a concise course in statistical inference*, Wasserman 2004  <a class="anchor" id="WAS04"></a>(see also errata in http://www.stat.cmu.edu/~larry/all-of-statistics/): **Chapter 8, 9**

* *Statistics in theory and Practice*, Lupton 1993 <a class="anchor" id="LUP93"></a>: **Chapter 6, 7, 8, 9**

* [Numerical recipes](http://www2.units.it/ipl/students_area/imm2/files/Numerical_Recipes.pdf) by Press et al. Cambridge University press: **Chapter 14**

* *Penn State Lectures on statistics*: https://onlinecourses.science.psu.edu/stat414/  ** STAT 415 Sect 6-7-8**

* *Penn State Lectures on statistics*: https://onlinecourses.science.psu.edu/stat200 **STAT 200 Sect. 8, 8, 9**

* *Scipy course on statistics* http://www.scipy-lectures.org/packages/statistics/index.html#id9

Other useful references to know more about the topics covered in this lecture: 

- Online Statistics Education: A Multimedia Course of Study (http://onlinestatbook.com/). Project Leader: David M. Lane, Rice University.

- Bootstrap: A. C. Davison, D. V. Hinkley and G. A. Young, Statistical science, 2003, 18, 2, 141: Recent Developments in Bootstrap Methodology https://projecteuclid.org/download/pdf_1/euclid.ss/1063994969

- Bootstrap: University of Kentucky: STAT 621 http://web.as.uky.edu/statistics/users/pbreheny/621/F12/ by Patrick Breheny 
    
- Fisher matrices and confidence ellipses: Coe 2009 https://arxiv.org/abs/0906.4123    

- Online Statistics Education: A Multimedia Course of Study (http://onlinestatbook.com/). Project Leader: David M. Lane, Rice University.
